In [ ]:
import json
import os
import matplotlib.pyplot as plt
import pandas as pd
import folium
import ipinfo
from tqdm import tqdm
import colorsys

In [ ]:
IPINFO_TOKEN = "8ec3bebb580b49"

handler = ipinfo.getHandler(IPINFO_TOKEN)


def get_coodinates(ip):
    details = handler.getDetails(ip)
    return [float(details.latitude), float(details.longitude)]

def get_as(ip):
    details = handler.getDetails(ip)
    return details.org

In [ ]:
def get_localisation(folders):
    output = {}
    for folder in folders:
        for subfolder in tqdm([f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f)) and f.startswith("rpm_")]):
            logfile = os.path.join(folder, subfolder, "test.json")
            if os.path.exists(logfile):
                try:
                    with open(logfile) as file:
                        data = json.load(file)
                        ip = data["clientIp"].split(" - ")[0]
                        if ip in output:
                            output[ip]["count"] += 1
                        else :
                            output[ip] = {
                                "count" : 1,
                                "coord": get_coodinates(ip),
                                "as" : get_as(ip)
                            }
                except:
                    pass
    return output

In [ ]:
output = get_localisation(["memoire-vps-output", "memoire-vps-output2"])


In [ ]:
sum([output[ip]["count"] for ip in output])

In [ ]:
coords = []
labels = []

for ip in output:
    for _ in range(output[ip]["count"]):
        coords.append(output[ip]["coord"])
        labels.append(output[ip]["as"])

In [ ]:
from folium.plugins import MarkerCluster

m2 = folium.Map(
    location=[50.668081, 4.6118324],
    tiles="cartodbpositron", 
    zoom_start=3
)

marker_cluster = MarkerCluster().add_to(m2)

for coord, label in zip(coords, labels):
    folium.Marker(location=coord, popup=str(label)).add_to(marker_cluster)

m2

In [ ]:
m2.save("map.html")
m2.save("map.pdf")
